In [5]:
import json
import os

In [2]:
gemini_models = ["gemini-1.0-pro-vision-001", "gemini-1.5-pro-preview-0409"]

In [6]:
gemini_models = ['gemini-1.5-flash-preview-0514']

In [7]:
dataset_name = "AR-LSAT"
split = "dev"
model_name = gemini_models[0]
backup = "random"

program_path = os.path.join("outputs/logic_programs", f"{dataset_name}_{split}_{model_name}.json")
result_path = os.path.join("outputs/logic_inference", f"{dataset_name}_{split}_{model_name}_backup-{backup}.json")

with open(program_path, "r") as json_file:
    programs = json.load(json_file)

with open(result_path, "r") as json_file:
    results = json.load(json_file)

In [8]:
dataset_name = "AR-LSAT"
split = "dev"
model_name = "gpt-4"
backup = "random"

program_path = os.path.join("outputs/logic_programs", f"{dataset_name}_{split}_{model_name}.json")
result_path = os.path.join("outputs/logic_inference", f"{dataset_name}_{split}_{model_name}_backup-{backup}.json")

with open(program_path, "r") as json_file:
    programs_gpt = json.load(json_file)

with open(result_path, "r") as json_file:
    results_gpt = json.load(json_file)


In [6]:
for program, result, program_gpt, result_gpt in zip(programs, results, programs_gpt, results_gpt):
    if result_gpt["flag"] == "success" and result["flag"] in ["parsing error", "execution error"]:
        print(program["raw_logic_programs"][0])
        break

In [7]:
for program, result in zip(programs, results):
    # if result["id"] ==  "ar_lsat_200006_1-G_1_6":
    #     print(result["flag"])
    # print(result["flag"])
    if result["flag"] == "success":
        print(result["flag"])
        print(program["raw_logic_programs"][0])
        break


In [3]:
from models.symbolic_solvers.z3_solver.sat_problem_solver import LSAT_Z3_Program

In [12]:
logic_program = '''# Declarations
people = EnumSort([Vladimir, Wendy])
meals = EnumSort([breakfast, lunch, dinner, snack])
foods = EnumSort([fish, hot_cakes, macaroni, omelet, poached_eggs])
eats = Function([people, meals] -> [foods])

# Constraints
ForAll([m:meals], eats(Vladimir, m) != eats(Wendy, m)) ::: At no meal does Vladimir eat the same kind of food as Wendy
ForAll([p:people, f:foods], Count([m:meals], eats(p, m) == f) <= 1) ::: Neither of them eats the same kind of food more than once during the day
ForAll([p:people], Or(eats(p, breakfast) == hot_cakes, eats(p, breakfast) == poached_eggs, eats(p, breakfast) == omelet)) ::: For breakfast, each eats exactly one of the following: hot cakes, poached eggs, or omelet
ForAll([p:people], Or(eats(p, lunch) == fish, eats(p, lunch) == hot_cakes, eats(p, lunch) == macaroni, eats(p, lunch) == omelet)) ::: For lunch, each eats exactly one of the following: fish, hot cakes, macaroni, or omelet
ForAll([p:people], Or(eats(p, dinner) == fish, eats(p, dinner) == hot_cakes, eats(p, dinner) == macaroni, eats(p, dinner) == omelet)) ::: For dinner, each eats exactly one of the following: fish, hot cakes, macaroni, or omelet
ForAll([p:people], Or(eats(p, snack) == fish, eats(p, snack) == omelet)) ::: For a snack, each eats exactly one of the following: fish or omelet
eats(Wendy, lunch) == omelet ::: Wendy eats an omelet for lunch

# Options
Question ::: Vladimir must eat which one of the following foods?
is_valid(Exists([m:meals], eats(Vladimir, m) == fish)) ::: (A)
is_valid(Exists([m:meals], eats(Vladimir, m) == hot_cakes)) ::: (B)
is_valid(Exists([m:meals], eats(Vladimir, m) == macaroni)) ::: (C)
is_valid(Exists([m:meals], eats(Vladimir, m) == omelet)) ::: (D)
is_valid(Exists([m:meals], eats(Vladimir, m) == poached_eggs)) ::: (E)'''


z3_program = LSAT_Z3_Program(logic_program, 'AR-LSAT')
# print(z3_program.standard_code)

output, error_message = z3_program.execute_program()
print(output)
# print(z3_program.answer_mapping(output))

['(D)']
